# 1. Imports

In [1]:
import sys, os, datetime, pickle, time
import string, pdb, tqdm
import random, keras, os.path, yaml
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
#
from functions import *
from models import *
from Builder import ConvNetBuilder
#
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
#
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
#
from timeit import default_timer as timer
from datetime import datetime, timedelta, date
from PIL import Image
from IPython.display import Image
# %matplotlib inline

#
tf.autograph.set_verbosity(0)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
# config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# tf.get_logger().setLevel('ERROR')

[]


# 2. Preparing Data

In [2]:
# Obtener la ruta del directorio tres niveles arriba
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))

# Cargar datasets desde el directorio tres niveles arriba
data_norm_path = os.path.join(parent_dir, 'Dataset/data_norm.pkl')

with open(data_norm_path, 'rb') as handle:
    GlblFrm = pickle.load(handle)
    dosD    = pickle.load(handle)
    coils   = pickle.load(handle)
    nclmaps = pickle.load(handle)


# 3. Building Sets

In [3]:
#  Selecting coils OK and NOK
lsids1 = coils.loc[coils['Label']==1,'SID'].tolist()
lsids2 = coils.loc[coils['Label']==2,'SID'].tolist()
#
# Select and extract 30 coil's sids for independent assessemnt
ass1 = random.sample(lsids1, 30)
ass2 = random.sample(lsids2, 30)
res1 = np.array(list(set(lsids1) - set(ass1)))
res2 = np.array(list(set(lsids2) - set(ass2)))
#
# The remaining are organized for building the model
random.shuffle(res1)
random.shuffle(res2)
train1, validate1, test1 = np.split(res1,[int(.7*len(res1)), int(.9*len(res1))])
train2, validate2, test2 = np.split(res2,[int(.7*len(res2)), int(.9*len(res2))])
#
# Building the full sets
train = train1.tolist() + train2.tolist()
valid = validate1.tolist() + validate2.tolist()
test  = test1.tolist()  + test2.tolist()
ass   = ass1   + ass2
#

In [4]:
def featureMap(id, nlcmaps):
    arr1 = nclmaps[id][1234]['nzne'].to_numpy()
    arr2 = nclmaps[id][1243]['nzne'].to_numpy()
    arr  = np.concatenate((arr1, arr2), axis=1)
    return(arr)
#
def prep_dataset(setd,nlcmaps):
    setd_f = []
    setd_l = []
    for i in setd:
        arrimg = featureMap(i, nclmaps)
        lbl = coils.loc[coils['SID']==i,'Label'].values[0]
        setd_f.append(arrimg)
        if lbl == 1:
            setd_l.append([1.,0.])
        if lbl == 2:
            setd_l.append([0.,1.])
    setd_f = np.array(setd_f)
    setd_l = np.array(setd_l)
    return([setd_f,setd_l])
#
def factory_rep(arr,step=0):
    # arr image havind 264 rows and 18 (9 face A+9 face B) columns of normalized data 
    # mirror per face over x axis
    # channels 0-3 => 5-8 and 5-8 => 0-3
    permut1 = list(range(8,4,-1))+[4] + list(range(3,-1,-1))+ list(range(17,13,-1))+[13]+list(range(12,8,-1))
    idx = np.empty_like(permut1)
    idx[permut1] = np.arange(len(permut1))
    arr1 = arr[:,idx]
    permut2 = list(range(263,131,-1))+ list(range(131,-1,-1))
    idx = np.empty_like(permut2)
    idx[permut2] = np.arange(len(permut2))    
    arr2 = arr[idx,:]
    arr3 = arr1[idx,:]
    res  = [arr, arr1, arr2, arr3]
    if step > 0:
        newa = arr
        end = arr.shape[0]-step
        for i in range(arr.shape[0] // step):
            permut = list(range(arr.shape[0]-step,arr.shape[0]))+list(range(0,end))
            idx = np.empty_like(permut)
            idx[permut] = np.arange(len(permut))
            newb= newa[idx,:]
            res.append(newb)
            newa= newb
    res = np.array(res)
    return(res)
#

def prep_dataset_aug(setd,nlcmaps,tlab=-1):
    setd_f = []
    setd_l = []
    for i in setd:
        arrimg = featureMap(i, nclmaps)
        lbl = coils.loc[coils['SID']==i,'Label'].values[0]
        if lbl == tlab: # if lower class => higher augmentation
            res = factory_rep(arrimg,step=8)
        else:
            res = factory_rep(arrimg)
        for j in range(res.shape[0]):
            setd_f.append(res[j,:,:])
            if lbl == 1:
                setd_l.append([1.,0.])
            if lbl == 2:
                setd_l.append([0.,1.])
    setd_f = np.array(setd_f)
    setd_l = np.array(setd_l)
    return([setd_f,setd_l])

In [5]:
goal = 'C:/Users/alexm/OneDrive/Escritorio/UNIVERSIDAD/TFG/ALVAREZ_CASTRO_Alejandro/ALVAREZ_CASTRO_Alejandro/ZN_1D_imgs/'
# for i in train + valid + test:
train_f, train_l = prep_dataset_aug(train,nclmaps,2)
np.savez(goal+'train.npz', features=train_f, labels=train_l)
#
valid_f, valid_l = prep_dataset(valid,nclmaps)
np.savez(goal+'validation.npz', features=valid_f, labels=valid_l)
#
test_f, test_l = prep_dataset(test,nclmaps)
np.savez(goal+'test.npz', features=test_f, labels=test_l)
#
ass_f, ass_l = prep_dataset(ass,nclmaps)
np.savez(goal+'assess.npz', features=ass_f, labels=ass_l)
#

In [6]:
#
npzfile = np.load(goal+'train.npz')
print(npzfile['features'].shape)
print(npzfile['labels'].shape)

(4293, 264, 18)
(4293, 2)


# 4. Building the model

In [7]:
# kernel_widths = [20, 0, 20, 20, 0, 6, 0, 6]  # Width of kernel for each layer (on the position where there is not a convolutional layer it must be 0)
# filters = [32, 0, 32, 32, 0, 32, 0, 32]  # Number of filters for each layer (on the position where there is not a convolutional layer it must be 0)
# dropouts = [0.4, 0.4]  # Dropout rate for each layer (At least as many as dropout layers)
# layer_types = ['C', 'N', 'C', 'C', 'P','C', 'N', 'C', 'P', 'flatten', 'F', 'D','F', 'D', 'F', 32, 16, 2] 

# builder = ConvNetBuilder(kernel_widths, filters, dropouts, layer_types)

# Cargar parámetros desde el archivo YAML
with open('config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

# Obtener los parámetros de la red neuronal
network_params = config['network_params']

# Crear una instancia de ConvNetBuilder con los parámetros cargados
builder = ConvNetBuilder(**network_params)



# Build the model
model, name = builder.build_model()

# 5. Training the model

In [8]:
num_simulations = 1

In [9]:
metrics_plot, conf_plot, mean_precision, std_dev_precision, mean_recall, std_dev_recall, mean_f1, std_dev_f1, mean_accuracy, std_dev_accuracy = simulations(name, num_simulations, model, train_f, train_l, valid_f, valid_l, ass_f, ass_l)

Epoch 1/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.4833 - loss: 1.0672 - val_accuracy: 0.1746 - val_loss: 0.7476
Epoch 2/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.6792 - loss: 0.5841 - val_accuracy: 0.1746 - val_loss: 0.8237
Epoch 3/100
 3/86 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7122 - loss: 0.5143

C:\Users\alexm\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\callbacks\model_checkpoint.py:199: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=self._current_epoch, batch=batch, logs=logs)


86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.7408 - loss: 0.5249 - val_accuracy: 0.1746 - val_loss: 0.9962
Epoch 4/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.7828 - loss: 0.4744 - val_accuracy: 0.1825 - val_loss: 1.0230
Epoch 5/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.8015 - loss: 0.4393 - val_accuracy: 0.1825 - val_loss: 0.8958
Epoch 6/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.8156 - loss: 0.4255 - val_accuracy: 0.4524 - val_loss: 0.8640
Epoch 7/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.8590 - loss: 0.3535 - val_accuracy: 0.5317 - val_loss: 0.7566
Epoch 8/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.8648 - loss: 0.3564 - val_accuracy: 0.7143 - val_loss: 0.6092
Epoch 9/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.8745 - loss: 0.3246 - val_accuracy: 0.7302 - val_loss: 0.5627
Epoch 10/100
86/86 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.8821 - loss: 0.3121 - val_accuracy: 0.7619 - val_

In [10]:
metrics_plot

In [11]:
conf_plot

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 245, 32)        │        11,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 245, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 226, 32)        │        20,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 207, 32)        │        20,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 103, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 98, 32)         │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 98, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 93, 32)         │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 46, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1472)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        47,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            34 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 338,392 (1.29 MB)

 Trainable params: 112,754 (440.45 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 225,510 (880.90 KB)

NoneType